In [1]:
import sys
sys.path.append('../')
from rocky.core import rocky
from rocky.triangle import Triangle
from rocky.model_selection.TriangleTimeSeriesSplit import TriangleTimeSeriesSplit
import pandas as pd
import numpy as np

In [ ]:
t = Triangle.from_taylor_ashe()
t.df

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Create a Rocky object
rky = rocky()

# add triangle
# rocky_inputs_file = r"O:\PARM\Individual folders\A Weaver\Reserving\rocky\rocky-inputs.xlsx"
# rky.FromExcel(filename=rocky_inputs_file, sheet_name='triangle', id='paid_loss')
rky.SampleTri('taylor_ashe')

# add model
rky.AddModel('icrfs', model_class='loglinear', tri=rky.paid_loss, use_cal=True, plot_width=None)

This is for testing and compatability only. Please do not assign much credibility to these estimates for the purposes selecting carried reserves.


In [ ]:
cv = TriangleTimeSeriesSplit(triangle=rky.icrfs.tri, model_type='loglinear', loglinear_grid=None)
# dir(cv)

['CalculateParameterParetoFront',
 'GetSplit',
 'OptimalParameters',
 'RunCrossValidation',
 'SetParameterGrid',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_fit_ward_clusters',
 'clustering_grid',
 'clustering_hyperparameters',
 'grid',
 'has_tuning_results',
 'is_tuned',
 'log_transform',
 'loglinear_grid',
 'model',
 'model_type',
 'n_failed_to_converge',
 'n_splits_',
 'randomforest_grid',
 'regression_hyperparameters',
 'split',
 'tie_criterion',
 'tri',
 'xgboost_grid']

In [ ]:
cv.tri.get_X_id()
# for s in cv.GetSplit():
#     print(s)

,tri,is_observed,accident_period,development_period,calendar_period
0,357848.0,1,2001,1,2001
1,352118.0,1,2002,1,2002
2,290507.0,1,2003,1,2003
3,310608.0,1,2004,1,2004
4,443160.0,1,2005,1,2005
...,...,...,...,...,...
95,NaN,0,2006,10,2015
96,NaN,0,2007,10,2016
97,NaN,0,2008,10,2017
98,NaN,0,2009,10,2018


In [ ]:
# set hyperparameters
# rky.icrfs.SetHyperparameters(alpha=0.05, l1_ratio=0.1)
rky.icrfs.TuneFitHyperparameters()

# fit model
rky.icrfs.Fit()

# basic hetero adjustment (one-step, no cross-validation,
# just one parameter per development period)
rky.icrfs.FitHetero()

In [ ]:
rky.icrfs.LassoFeatureImportance()

,0.0000,0.0025,0.0050,0.0075,0.0100,0.0125,0.0150,0.0175,0.0200,0.0225,...,0.3575,0.3600,0.3625,0.3650,0.3675,0.3700,0.3725,0.3750,0.3775,0.3800
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
1,0.479775,0.360861,0.306763,0.271149,0.235931,0.200019,0.168639,0.141931,0.112191,0.081354,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000,-0.000000,-0.000000,-0.0
2,0.359902,0.175055,0.116048,0.081324,0.048298,0.016022,0.000000,0.000000,0.000000,0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000,-0.000000,-0.000000,-0.0
3,0.186261,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000,-0.000000,-0.000000,-0.0
4,0.298483,0.041897,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
5,0.372169,0.055070,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
6,0.476623,0.119138,0.059150,0.010231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
7,0.607685,0.193406,0.126456,0.062070,0.008888,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
8,0.287072,-0.034097,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
9,0.094751,-0.172006,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0


In [ ]:
rky.icrfs.Ultimate()

accident_period
2001-01-01   -6.0
2002-01-01   -2.0
2003-01-01   -1.0
2004-01-01   -0.0
2005-01-01    2.0
2006-01-01    3.0
2007-01-01    5.0
2008-01-01    7.0
2009-01-01    7.0
2010-01-01    8.0
Name: Log-linear Ultimate, dtype: float64

In [ ]:
p = rky.icrfs.GetX('train').columns.to_frame().reset_index(drop=True)
p.columns = ['feature']
df.join(p, how='left').set_index('feature').T.plot(kind='line')

In [ ]:
still_tuning = df.iloc[:,0].ne("abc")
first_col = still_tuning.astype(int)

def stop_condition(df, col):
    return df[col].eq(0) & still_tuning

for i, col in enumerate(df.columns.tolist()):
    first_col = first_col.mask(stop_condition(df, col), col)
    still_tuning = still_tuning.mask(stop_condition(df, col), False)

        
sorted = first_col.sort_values(ascending=True)
# sorted
out = pd.DataFrame(sorted)
out.columns = ['alpha causing stop']

out.join(rky.icrfs.GetX().columns.to_frame().reset_index(drop=True), how='left')

,alpha causing stop,0
0,0.0000,intercept
34,0.0000,calendar_period_2017
33,0.0000,calendar_period_2016
32,0.0000,calendar_period_2015
31,0.0000,calendar_period_2014
30,0.0000,calendar_period_2013
29,0.0000,calendar_period_2012
28,0.0000,calendar_period_2011
35,0.0000,calendar_period_2018
36,0.0000,calendar_period_2019


In [ ]:
rky.icrfs.tri.ata().agg(['min', 'max', 'mean', 'std'], axis=0).round(3).T.plot(kind='line')

In [ ]:
import numpy as np
import pandas as pd



In [ ]:
rky.icrfs.GetX('train').shape

(55, 37)

In [ ]:
rky.icrfs.TrendPlot('cal')

Calendar Period
Calendar Period
